In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader,TensorDataset
import torch.utils.data as data
import numpy as np
import scipy.io as sio
import math
import argparse
import random
import os
from My_Loss import HardTripletLoss
from tensorboardX import SummaryWriter

In [2]:
BATCH_SIZE = 64
EPISODE = 200000
TEST_EPISODE = 1000
LEARNING_RATE =1e-5
Weight_Deacy = 0
GPU = 0
Margin = 3

In [3]:
print("init dataset")
##################################参数##################################################################
dataroot = '../data'
dataset = 'AwA1_data'
image_embedding = 'res101'               #ResNet101层
class_embedding = 'original_att'         #属性表达 85-d
#######################################读取视觉特征###################################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + image_embedding + ".mat")  #scipy loadmat
 
feature = matcontent['features'].T         #转置 30478x2048 每一行是一个完整的样本

label = matcontent['labels'].astype(int).squeeze() - 1   #matlab begin 1 ,numpy begin 0
########################################读取属性特征###########################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + class_embedding + "_splits.mat")

    
# numpy array index starts from 0, matlab starts from 1
trainval_loc = matcontent['trainval_loc'].squeeze() - 1    #squeeze()去掉维度中的1 AxBx1 --->AxB

test_seen_loc = matcontent['test_seen_loc'].squeeze() - 1
test_unseen_loc = matcontent['test_unseen_loc'].squeeze() - 1

attribute = matcontent['att'].T    #转置 50x85 每行是整个属性向量

x = feature[trainval_loc]                      # train_features trainval里面是图片的编号 begin with 0 ，19832个
train_label = label[trainval_loc].astype(int)  # train_label  int类型没变 每个图片的lable 19832个
train_id = np.unique(train_label)

att = attribute[train_label]                   # train attributes 每个图片的属性 19832个


    
x_test = feature[test_unseen_loc]                   # test_feature 5685个
test_label = label[test_unseen_loc].astype(int)     # test_label   5685个

x_test_seen = feature[test_seen_loc]                #test_seen_feature 4958个
test_label_seen = label[test_seen_loc].astype(int)  # test_seen_label  4958个
    
test_id = np.unique(test_label)                     # test_id  10个类 ，unique去重
att_pro = attribute[test_id]                        # test_attribute 每一类的属性向量 10x85

# train set
train_features = torch.from_numpy(x)   #np-->tensor
#train_fearures_negative = torch.from_numpy(x_negative)

sample_attributes=[]
train_label = torch.from_numpy(train_label).unsqueeze(1) #每张图片的属性转化 ，unsqueeze(1)就是插入到第一维度 AxB维-->Ax1xB
# attributes
all_attributes = np.array(attribute)  #所有50类属性转变为numpy数组???属性向量仍然用的numpy类型 没有转化为pytorch
    
#print('-'*50)
attributes = torch.from_numpy(attribute) 
#print(attribute)
# test set

test_features = torch.from_numpy(x_test)
#print(test_features.shape)

test_label = torch.from_numpy(test_label).unsqueeze(1)
#print(test_label.shape)

testclasses_id = np.array(test_id)
#print(testclasses_id.shape)

test_attributes = torch.from_numpy(att_pro).float()
#print(test_attributes.shape)

test_seen_features = torch.from_numpy(x_test_seen)
#print(test_seen_features.shape)

test_seen_label = torch.from_numpy(test_label_seen)

train_data = TensorDataset( train_label, train_features )
#train_data = TensorDataset(train_label, train_features, train_fearures_negative)

#################here need new code to make triplet data#####################
print('-'*100)

init dataset
----------------------------------------------------------------------------------------------------


In [4]:
from my_net_3 import AttributeNetwork
from my_net_3 import MetricNetwork

In [5]:
# init network
print("init networks")
attribute_network = AttributeNetwork(85,1024,2048)  #85d属性 1024隐藏层 2048输出 85d到2048d
metric_network = MetricNetwork(2048,1024,2048)

attribute_network.cuda(GPU) 
metric_network.cuda(GPU)

#学习率每200k步 乘0.5
attribute_network_optim = torch.optim.Adam(attribute_network.parameters(), lr=LEARNING_RATE,weight_decay=Weight_Deacy)
metric_network_optim = torch.optim.Adam(metric_network.parameters(), lr=LEARNING_RATE,weight_decay=Weight_Deacy)
#
#triplet_network_optim = torch.optim.SGD(triplet_network.parameters(), lr=LEARNING_RATE,momentum=0.9 , 
#                                         weight_decay=Weight_Deacy)

attribute_network_scheduler = StepLR(attribute_network_optim , step_size=20000 , gamma=0.5)
metric_network_scheduler = StepLR(metric_network_optim , step_size=20000 , gamma=0.5)
#
print('-'*100)

init networks
----------------------------------------------------------------------------------------------------


In [6]:
def compute_accuracy(test_features, test_label, test_id, test_attributes):
    
    test_data = TensorDataset(test_features, test_label)
    test_batch = 32
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    total_rewards = 0

    sample_labels = test_id
    sample_attributes = test_attributes
    class_num = sample_attributes.shape[0]
    test_size = test_features.shape[0]

    print("class num:", class_num)

    for batch_features,batch_labels in test_loader:

        batch_size = batch_labels.shape[0]
        batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(class_num, 0))
        batch_features_ext = metric_network(Variable(batch_features_ext).cuda(GPU).float())  # 32*1024


        sample_features = metric_network(attribute_network(Variable(sample_attributes).cuda(GPU).float()))
        sample_features_ext = sample_features.repeat(batch_size, 1)


        relations = F.pairwise_distance(batch_features_ext, sample_features_ext, 2).view(-1, class_num)
        re_batch_labels = []
        for label in batch_labels.numpy():
            index = np.argwhere(sample_labels == label)
            re_batch_labels.append(index[0][0])
        re_batch_labels = torch.cuda.LongTensor(re_batch_labels)


        _, predict_labels = torch.min(relations.data, 1)
        rewards = [1 if predict_labels[j] == re_batch_labels[j] else 0 for j in range(batch_size)]
        total_rewards += np.sum(rewards)
    test_accuracy = total_rewards/1.0/test_size
    return  test_accuracy

In [7]:
def compute_accuracy_per_class(test_features, test_label, test_id, test_attributes):
    
    test_data = TensorDataset(test_features, test_label)
    test_batch = 32
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    total_rewards = 0
    #print(test_features.size())
    sample_labels = test_id
    sample_attributes = test_attributes
    class_num = sample_attributes.shape[0]
    test_size = test_features.shape[0]
    per_class_num = np.zeros(class_num)
    per_class_true= np.zeros(class_num)

    print("class num:", class_num)

    for batch_features,batch_labels in test_loader:

        batch_size = batch_labels.shape[0]
        batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(class_num, 0))
        batch_features_ext = metric_network(Variable(batch_features_ext).cuda(GPU).float())  # 32*1024


        sample_features = metric_network(attribute_network(Variable(sample_attributes).cuda(GPU).float()))
        sample_features_ext = sample_features.repeat(batch_size, 1)


        relations = F.pairwise_distance(batch_features_ext, sample_features_ext, 2).view(-1, class_num)
        re_batch_labels = []
        for label in batch_labels.numpy():
            index = np.argwhere(sample_labels == label)
            re_batch_labels.append(index[0][0])
        re_batch_labels_id, batch_per_num = np.unique(re_batch_labels , return_counts=True) 
        re_batch_labels = torch.cuda.LongTensor(re_batch_labels)
        
        for each in range(re_batch_labels_id.size):
            #print(re_batch_labels_id[each])
            #print(batch_per_num[each])
            per_class_num[re_batch_labels_id[each]] = per_class_num[re_batch_labels_id[each]] + batch_per_num[each]
        #print(re_batch_labels_id)
        #print('-'*100)
        #print(batch_per_num)
        #print('-'*100)


        _, predict_labels = torch.min(relations.data, 1)
        for j in range(batch_size):
            if predict_labels[j] == re_batch_labels[j]:
                per_class_true[re_batch_labels[j]] = per_class_true[re_batch_labels[j]] + 1
            
        
        
        rewards = [1 if predict_labels[j] == re_batch_labels[j] else 0 for j in range(batch_size)]
        total_rewards += np.sum(rewards)
    per_accuracy = per_class_true[np.nonzero(per_class_num)] / per_class_num[np.nonzero(per_class_num)]
    

    test_accuracy = np.sum(per_accuracy)/1.0/np.count_nonzero(per_class_num)
    test_accuracy2 = total_rewards/1.0/test_size
    #print(np.count_nonzero(per_class_num))
    return  test_accuracy, test_accuracy2

In [8]:
print("training...")
last_accuracy = 0.0
loss_zero_num =0
#writer = SummaryWriter()
for episode in range(EPISODE):

    attribute_network_scheduler.step(episode)
    metric_network_scheduler.step(episode)

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

    batch_labels, batch_features = train_loader.__iter__().next()
    batch_id = np.unique(batch_labels)

    batch_attributes = torch.Tensor([all_attributes[i] for i in batch_id]).squeeze(1)
    batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(batch_id.size, 0))
    batch_attributes_ext = batch_attributes.repeat(BATCH_SIZE, 1)

    batch_features_ext = Variable(batch_features_ext).cuda(GPU).float()  # 32*2048
    batch_attributes_ext = Variable(batch_attributes_ext).cuda(GPU)


    re_batch_labels = []
    for label in batch_labels.numpy():
        index = np.argwhere(batch_id == label)
        re_batch_labels.append(index[0][0])
    re_batch_labels = torch.cuda.LongTensor(re_batch_labels)
    re_batch_labels = Variable(re_batch_labels).cuda(GPU)
    

    criterion = HardTripletLoss(margin = Margin).cuda(GPU)
    triplet_loss= criterion(metric_network(attribute_network(batch_attributes_ext)), 
                            metric_network(batch_features_ext), re_batch_labels)
    if triplet_loss == 0:
        loss_zero_num = loss_zero_num + 1
    
    
    attribute_network.zero_grad()
    metric_network.zero_grad()
    
    triplet_loss.backward()
    
    attribute_network_optim.step()
    metric_network_optim.step()

    
    if (episode+1)%200 == 0 or episode==0:
        print("episode:", episode+1, "loss", triplet_loss)
        print('loss_zero_number= ',loss_zero_num)
        loss_zero_num = 0
        #writer.add_scalar('data/loss', triplet_loss, episode)
    if (episode+1)%200 == 0 or episode==0:
        print("Testing...")
        #attribute_network.eval()
        zsl_accuracy, zsl_accuracy2 = compute_accuracy_per_class(test_features, test_label, test_id, test_attributes)
        gzsl_unseen_accuracy , gzsl_unseen_accuracy2= compute_accuracy_per_class(test_features, test_label, np.arange(50), attributes)
        gzsl_seen_accuracy ,gzsl_seen_accuracy2 = compute_accuracy_per_class(test_seen_features, test_seen_label, np.arange(50), attributes)
        H = 2 * gzsl_seen_accuracy * gzsl_unseen_accuracy / (gzsl_unseen_accuracy + gzsl_seen_accuracy)
        H2 = 2 * gzsl_seen_accuracy2 * gzsl_unseen_accuracy2 / (gzsl_unseen_accuracy2 + gzsl_seen_accuracy2)
        print('zsl:', zsl_accuracy)
        #print('zsl:', zsl_accuracy2)
        print('gzsl: unseen=%.4f , seen=%.4f , h=%.4f' % (gzsl_unseen_accuracy , gzsl_seen_accuracy, H))
        #print('gzsl: unseen=%.4f , seen=%.4f , h=%.4f' % (gzsl_unseen_accuracy2 , gzsl_seen_accuracy2, H2))
        print('_'*100)
        #writer.add_scalar('data/zsl_accuracy', zsl_accuracy, episode)
        #writer.add_scalar('data/gzsl_unseen_accuracy', gzsl_unseen_accuracy, episode)
        #writer.add_scalar('data/gzsl_seen_accuracy', gzsl_seen_accuracy, episode)
        #writer.add_scalar('data/H', H, episode)
#writer.export_scalars_to_json("./test.json")
#writer.close()

training...
episode: 1 loss tensor(3.0837, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.11377845580631751
gzsl: unseen=0.0000 , seen=0.0246 , h=0.0000
____________________________________________________________________________________________________
episode: 200 loss tensor(2.9845, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.20042278576947342
gzsl: unseen=0.0124 , seen=0.0808 , h=0.0214
____________________________________________________________________________________________________
episode: 400 loss tensor(2.9451, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.3152577916456022
gzsl: unseen=0.0351 , seen=0.1782 , h=0.0587
____________________________________________________________________________________________________
episode: 600 loss tensor(2.8835, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num:

episode: 5600 loss tensor(2.1254, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6183153694027589
gzsl: unseen=0.4868 , seen=0.7329 , h=0.5850
____________________________________________________________________________________________________
episode: 5800 loss tensor(2.0856, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6467922116824697
gzsl: unseen=0.5115 , seen=0.7301 , h=0.6016
____________________________________________________________________________________________________
episode: 6000 loss tensor(1.4042, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6661945971927103
gzsl: unseen=0.5229 , seen=0.7294 , h=0.6091
____________________________________________________________________________________________________
episode: 6200 loss tensor(1.9243, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
clas

episode: 11200 loss tensor(1.9739, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6519286250604405
gzsl: unseen=0.4917 , seen=0.7815 , h=0.6036
____________________________________________________________________________________________________
episode: 11400 loss tensor(1.6044, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6333977345160031
gzsl: unseen=0.4834 , seen=0.7836 , h=0.5979
____________________________________________________________________________________________________
episode: 11600 loss tensor(1.0160, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6579338993114432
gzsl: unseen=0.4874 , seen=0.7822 , h=0.6006
____________________________________________________________________________________________________
episode: 11800 loss tensor(1.9894, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50


episode: 16800 loss tensor(1.1759, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6260363837996031
gzsl: unseen=0.5090 , seen=0.7648 , h=0.6113
____________________________________________________________________________________________________
episode: 17000 loss tensor(1.9055, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.62829167036805
gzsl: unseen=0.4858 , seen=0.7682 , h=0.5952
____________________________________________________________________________________________________
episode: 17200 loss tensor(0.6882, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6184733808798801
gzsl: unseen=0.4736 , seen=0.7797 , h=0.5893
____________________________________________________________________________________________________
episode: 17400 loss tensor(1.3158, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
cl

episode: 22400 loss tensor(2.0464, device='cuda:0')
loss_zero_number=  1
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6522129881236806
gzsl: unseen=0.5024 , seen=0.7799 , h=0.6111
____________________________________________________________________________________________________
episode: 22600 loss tensor(0.6206, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6432124379349549
gzsl: unseen=0.4965 , seen=0.7850 , h=0.6083
____________________________________________________________________________________________________
episode: 22800 loss tensor(1.3585, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6447320272764603
gzsl: unseen=0.4852 , seen=0.7846 , h=0.5996
____________________________________________________________________________________________________
episode: 23000 loss tensor(1.7951, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50


...

episode: 95200 loss tensor(1.1937, device='cuda:0')
loss_zero_number=  3
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6106308591452119
gzsl: unseen=0.4686 , seen=0.7938 , h=0.5893
____________________________________________________________________________________________________
episode: 95400 loss tensor(0.4090, device='cuda:0')
loss_zero_number=  2
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6129138764536884
gzsl: unseen=0.4694 , seen=0.7931 , h=0.5898
____________________________________________________________________________________________________
episode: 95600 loss tensor(1.6676, device='cuda:0')
loss_zero_number=  2
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6126679709201942
gzsl: unseen=0.4639 , seen=0.7919 , h=0.5851
____________________________________________________________________________________________________
episode: 95800 loss tensor(0.6642, device='cuda:0')
loss_zero_number=  1
Testing...
class num: 10
class num: 50


episode: 100800 loss tensor(2.0215, device='cuda:0')
loss_zero_number=  1
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6115505303025732
gzsl: unseen=0.4668 , seen=0.7908 , h=0.5870
____________________________________________________________________________________________________
episode: 101000 loss tensor(1.0309, device='cuda:0')
loss_zero_number=  1
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6098839303914969
gzsl: unseen=0.4679 , seen=0.7908 , h=0.5880
____________________________________________________________________________________________________
episode: 101200 loss tensor(0.8600, device='cuda:0')
loss_zero_number=  1
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.610572828554171
gzsl: unseen=0.4675 , seen=0.7908 , h=0.5876
____________________________________________________________________________________________________
episode: 101400 loss tensor(0.9379, device='cuda:0')
loss_zero_number=  2
Testing...
class num: 10
class num: 

episode: 106400 loss tensor(0.9484, device='cuda:0')
loss_zero_number=  5
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6059498963002986
gzsl: unseen=0.4632 , seen=0.7914 , h=0.5844
____________________________________________________________________________________________________
episode: 106600 loss tensor(1.6646, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6069640475865594
gzsl: unseen=0.4611 , seen=0.7907 , h=0.5825
____________________________________________________________________________________________________
episode: 106800 loss tensor(0.5748, device='cuda:0')
loss_zero_number=  3
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.606486868134577
gzsl: unseen=0.4618 , seen=0.7910 , h=0.5832
____________________________________________________________________________________________________
episode: 107000 loss tensor(2.0625, device='cuda:0')
loss_zero_number=  2
Testing...
class num: 10
class num: 

episode: 112000 loss tensor(0.6894, device='cuda:0')
loss_zero_number=  2
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.607971585059946
gzsl: unseen=0.4634 , seen=0.7924 , h=0.5848
____________________________________________________________________________________________________
episode: 112200 loss tensor(2.1728, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6070133837238011
gzsl: unseen=0.4641 , seen=0.7931 , h=0.5855
____________________________________________________________________________________________________
episode: 112400 loss tensor(1.3523, device='cuda:0')
loss_zero_number=  2
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6075509395289295
gzsl: unseen=0.4637 , seen=0.7941 , h=0.5855
____________________________________________________________________________________________________
episode: 112600 loss tensor(0.5868, device='cuda:0')
loss_zero_number=  2
Testing...
class num: 10
class num: 

KeyboardInterrupt: 